In [1]:
import numpy as np
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import scanpy as sc
import scipy

In [2]:
adata = ad.read_h5ad("/Users/apple/Desktop/KB/data/BiddyData/biddy_6534_2000_norm_log.h5ad")

In [3]:
adata.shape

(6534, 2000)

In [4]:
len(adata.obs["clone_id"].unique())

169

In [4]:
adata.obs

timecourse reprogramming_day reprogramming   cell_type  \
HF1_AAAGCAATCCAGATCA_5          1                12           NaN   Ambiguous   
HF1_AACACGTAGAACAACT_5          1                12           NaN   Ambiguous   
HF1_AAACCTGAGAGCAATT_5          1                12           NaN  Fibroblast   
HF1_AACACGTTCGAGGTAG_5          1                12         False         iEP   
HF1_AACCATGGTCTAGAGG_5          1                12           NaN         iEP   
...                           ...               ...           ...         ...   
HF2_TTCTTAGAGCGACGTA_4          2                 9           NaN         iEP   
HF2_TTCCCAGGTGATGATA_4          2                 9           NaN         iEP   
HF2_TTCTCCTCATGAACCT_4          2                 9           NaN         iEP   
HF2_TTGGAACCAAGAAAGG_4          2                 9           NaN   Ambiguous   
HF2_TTGTAGGGTCAGAGGT_4          2                 9           NaN  Fibroblast   

                       cell_cycle cluster monocle_state  pseudotime  \
HF1_AAAGCAATCCAGATCA_5         G1       5             2    0.426868   
HF1_AACACGTAGAACAACT_5          S       5             2    0.549377   
HF1_AAACCTGAGAGCAATT_5        G2M       2             2    0.664971   
HF1_AACACGTTCGAGGTAG_5         G1       2             2    0.730338   
HF1_AACCATGGTCTAGAGG_5         G1       2             2    0.568279   
...                           ...     ...           ...         ...   
HF2_TTCTTAGAGCGACGTA_4        G2M       5             2    0.509046   
HF2_TTCCCAGGTGATGATA_4        G2M       5             2    0.530828   
HF2_TTCTCCTCATGAACCT_4        G2M       5             2    0.403314   
HF2_TTGGAACCAAGAAAGG_4         G1       2             2    1.029464   
HF2_TTGTAGGGTCAGAGGT_4         G1       5             2    0.532476   

                        CellTagD0_85k  CellTagD3_85k  CellTagD13_85k  \
HF1_AAAGCAATCCAGATCA_5          424.0          476.0             NaN   
HF1_AACACGTAGAACAACT_5          195.0            NaN             NaN   
HF1_AAACCTGAGAGCAATT_5          647.0          309.0             NaN   
HF1_AACACGTTCGAGGTAG_5          487.0          240.0             NaN   
HF1_AACCATGGTCTAGAGG_5          652.0          203.0             NaN   
...                               ...            ...             ...   
HF2_TTCTTAGAGCGACGTA_4         2657.0            NaN             NaN   
HF2_TTCCCAGGTGATGATA_4         2367.0         2514.0             NaN   
HF2_TTCTCCTCATGAACCT_4         2490.0         2042.0             NaN   
HF2_TTGGAACCAAGAAAGG_4         2764.0         2525.0             NaN   
HF2_TTGTAGGGTCAGAGGT_4         2294.0         2118.0             NaN   

                        CellTagD0_48k  CellTagD3_48k  CellTagD13_48k  \
HF1_AAAGCAATCCAGATCA_5          424.0          476.0             NaN   
HF1_AACACGTAGAACAACT_5          195.0            NaN             NaN   
HF1_AAACCTGAGAGCAATT_5          647.0          309.0             NaN   
HF1_AACACGTTCGAGGTAG_5          487.0          240.0             NaN   
HF1_AACCATGGTCTAGAGG_5          652.0          203.0             NaN   
...                               ...            ...             ...   
HF2_TTCTTAGAGCGACGTA_4         2657.0            NaN             NaN   
HF2_TTCCCAGGTGATGATA_4         2367.0         2514.0             NaN   
HF2_TTCTCCTCATGAACCT_4         2490.0         2042.0             NaN   
HF2_TTGGAACCAAGAAAGG_4         2764.0         2525.0             NaN   
HF2_TTGTAGGGTCAGAGGT_4         2294.0         2118.0             NaN   

                        clone_id  n_genes  
HF1_AAAGCAATCCAGATCA_5     424.0     4652  
HF1_AACACGTAGAACAACT_5     195.0     4222  
HF1_AAACCTGAGAGCAATT_5     647.0     3862  
HF1_AACACGTTCGAGGTAG_5     487.0     3198  
HF1_AACCATGGTCTAGAGG_5     652.0     3224  
...                          ...      ...  
HF2_TTCTTAGAGCGACGTA_4    2657.0     2038  
HF2_TTCCCAGGTGATGATA_4    2367.0     3610  
HF2_TTCTCCTCATGAACCT_4    2490.0     3993  
HF2_TTGGAACCAAGAAAGG_4    2

In [5]:
# Step 1: Identify unique clone_ids and their frequencies
clone_id_counts = adata.obs['clone_id'].value_counts()

In [6]:
# Step 2: Initialize empty lists to hold indices for adata_train and adata_test
test_indices = []
train_indices = []

# Step 3: Loop through each clone_id and split based on the criteria
for clone_id, count in clone_id_counts.items():
    clone_indices = adata.obs[adata.obs['clone_id'] == clone_id].index
    
    if count >= 10:
        # Select 10% of the elements randomly for adata_test
        test_size = int(np.ceil(0.1 * count))
        test_clone_indices = np.random.choice(clone_indices, size=test_size, replace=False)
        test_indices.extend(test_clone_indices)
        
        # The remaining elements go to adata_train
        train_clone_indices = list(set(clone_indices) - set(test_clone_indices))
        train_indices.extend(train_clone_indices)
    else:
        # All elements go to adata_train
        train_indices.extend(clone_indices)



In [7]:

# Step 4: Subset the original adata to create adata_train and adata_test
adata_train = adata[train_indices, :].copy()
adata_test = adata[test_indices, :].copy()


In [8]:
adata_test.shape, adata_train.shape

((641, 2000), (5893, 2000))

In [9]:
adata_test.write_h5ad('Biddy_test.h5ad')
adata_train.write_h5ad('Biddy_train.h5ad')



In [10]:
adata_test.obs["clone_id"].value_counts()

clone_id
493.0     131
2352.0     66
487.0      37
666.0      33
2721.0     30
         ... 
2756.0      1
2589.0      1
349.0       1
601.0       1
2819.0      1
Name: count, Length: 87, dtype: int64

In [11]:
adata_train.obs["clone_id"].value_counts()

clone_id
493.0     1178
2352.0     591
487.0      329
666.0      296
2721.0     263
          ... 
2902.0       5
2951.0       5
2863.0       5
2894.0       5
2367.0       5
Name: count, Length: 169, dtype: int64